# ## 1. Setup and Imports
# This section installs and imports the required libraries and modules.

In [1]:
from cnn import *
from loss import *
from validation import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt


# ## 2. Data Preparation
# This section handles data loading and transformations.

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

data_dir = "./data"
train_dataset = datasets.CIFAR10(root=data_dir, train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

Files already downloaded and verified


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ## 3. Model Initialization
# Initialize the model, loss function, and optimizer.

In [4]:

class CNN(nn.Module):
    def __init__(self,batch_size,device):
        super(CNN, self).__init__()
        self.conv1 = Conv2d(3, 16, kernel_size=3, stride=1, padding=1,device = device)
        # conv_params = {'kernel_size': 3, 'stride': 1, 'padding': 1, 'out_channels': 16}
        conv_shape = calculate_output_shape((3,32,32),"conv",layer_params={'kernel_size':3,'stride':1,'padding':1,'out_channels':16})
        self.relu = ReLU()
        self.pool = Pool2d(kernel_size=2, stride=2,pool = "max")
        pool_shape = calculate_output_shape(conv_shape,"maxpool",layer_params={'kernel_size':2,'stride':2,'padding':0})
        self.fc1 = FC(batch_size,pool_shape[0]* pool_shape[1] * pool_shape[2], 10,device = device)
        self.softmax = Softmax()
        self.layers = nn.ModuleList([self.conv1, self.relu, self.pool, self.fc1,self.softmax][::-1])

    def forward(self, x):
        x = (x - x.mean()) / (x.std() + 1e-5)  # explicit normalization
        x,_ = self.conv1.custom_forward(x)
        x = self.relu.custom_forward(x)
        x = self.pool.custom_forward(x)
        x = self.fc1.custom_forward(x)
        x  = self.softmax.custom_forward(x,dim = -1)
        return x,self.layers

In [5]:
device

device(type='cuda')

In [6]:

model = CNN(batch_size=8,device = device)
criterion = nn.CrossEntropyLoss()
loss_module = Loss(criterion)
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Validate Custom Backpropagation Implementation

In [7]:
sample_input = torch.randn(4, 3, 32, 32).half().to(device)  # Replace dimensions as per your network
sample_target = torch.randint(0, 10, (4,)).to(device)  # Replace target dimensions as needed
validate_backward(model, sample_input, sample_target,loss_module,device= device)


Backward pass validation: PASSED!


## Validate Custom Forward Implementation

In [9]:
sample_input = torch.randn(4, 3, 32, 32).half().to(device)  # Replace dimensions as per your network
validate_forward(model, sample_input,device =device)


Forward pass validation: PASSED!


## Train the model with pytorch optimizer 

In [ ]:
losses = train_with_optimizer(model, train_loader, nn.CrossEntropyLoss(), num_epochs=5,device=device)
